
<br>

<div style="text-align: center">
<h1>COMPUTO DE ALTO DESEMPEÑO</h1>
<br>
<br>

Roides Javier Cruz Lara <br><br>

LANIA-MCA, 2017 <br><br>

Instalación y Primeros pasos con Ipyparallel 
<br>

</div>


In [1]:
import numpy as np
print (np.__version__)
import pandas as pd
print (pd.__version__)

import matplotlib as plt
print (plt.__version__)

import ipyparallel as ipp
print (ipp.__version__)


1.11.1
0.18.1
2.0.2
6.0.2


# Creando un cliente

In [8]:
cliente = ipp.Client()

# Motores 

In [9]:
cliente.ids

[0, 1, 2, 3]

In [10]:
cliente

In [17]:
cliente[:]

<DirectView [0, 1, 2, 3]>

In [19]:
cliente[0].apply_sync(lambda : "Hola mundo") # utilizando solo un motor

'Hola mundo'

In [21]:
cliente[:].apply_sync(lambda : "Hola muchos alumnos") # utilizando todos los motores

['Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos']

In [23]:
dview = cliente[:] # guardando todos los motores a la variable dview
dview.apply_sync(lambda : "Hola Mundo")

['Hola Mundo', 'Hola Mundo', 'Hola Mundo', 'Hola Mundo']

In [24]:
dview

<DirectView [0, 1, 2, 3]>

In [25]:
import os
dview.apply_sync(os.getpid)

[8080, 11976, 8960, 11628]

In [26]:
import socket
dview.apply_sync(socket.gethostname)

['DESKTOP-033P0J6', 'DESKTOP-033P0J6', 'DESKTOP-033P0J6', 'DESKTOP-033P0J6']

# Utilización de las Vistas
Sumando 10 conjuntos de 3 numeros en paralelo usando todos los motores en la vista

In [27]:
dview.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

sumando 10 conjuntos de 3 numeros en paralelo usando motores alternados

In [30]:
cliente[::2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Sumando  10 conjuntos de 3 numeros usando un motor especifico

In [32]:
cliente[2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

# Balance de Carga

In [35]:
lview = cliente.load_balanced_view()
lview.map_sync(lambda x: sum(x), np.random.random((10,1000000)))

[500137.66064430156,
 500138.01832927956,
 500283.20606664184,
 500376.32916456781,
 500082.2501148649,
 499778.90699700877,
 499976.32898605592,
 500607.9803416502,
 499674.68500899465,
 500097.48229180893]

# Map
Permite aplicar cualquier funcion elemento a elemento a una secuencia. Para este ejemplo utilizaremos una funcion muy sencilla definida a continuacion.

In [36]:
def cube(x):
    return x**3
cube(2)

8

In [37]:

lista1=[cube(x) for x in range(1,7)]
lista1

[1, 8, 27, 64, 125, 216]

In [40]:
map(cube, range(1,7))

In [41]:
serial = map(cube, range(1,70))

In [42]:
parallel = dview.map_sync(cube, range(1,70))

In [43]:
serial == parallel

False

In [44]:
%timeit map(cube, range(1,70))
%timeit dview.map_sync(cube, range(1,70))

The slowest run took 6.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.19 µs per loop
10 loops, best of 3: 108 ms per loop


In [45]:
dview.map_sync(cube, range(1,7))


[1, 8, 27, 64, 125, 216]

In [46]:
def multi(a,b):
    return a*b
dview.map_sync(multi, [0,1,3,5],[2,4,6,8])

[0, 4, 18, 40]

In [47]:
# Una funcion para determinar si un numero dado es primo:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1

In [48]:
PrimeQ(2)

True

In [49]:
PrimeQ(137)

True

In [50]:
PrimeQ(135)

False